In [ ]:
#default_exp sources.core

# Source Interface

> The interface used by the sources for our audio data.

In [ ]:
#|export

import pandas as pd
import os
import xxhash
import shutil
import stringcase

# These are based on what I find in the data
AUDIO_EXTENSIONS = (".wav")

In [ ]:
#export

class Source:
    """All sources should implement this class."""
    @property
    def name(self) -> str:
        raise NotImplementedError()
    
    def get_files(self, root_dir: str) -> list[tuple[str, str]]:
        """List of paths and files to process, as returned by `get_filenames`."""
        raise NotImplementedError()
    
    def get_category(self, path: str, filename: str) -> str:
        """Category name for the given file (most sources hardcode a single category)."""
        raise NotImplementedError()
    
    def get_labels(self, path: str, filename: str) -> list[str]:
        """List of labels for the given file."""
        raise NotImplementedError()
    
    def get_extra(self, path: str, filename: str) -> str:
        """(Optional) Extra information for the given file."""
        return None

In [ ]:
#hide_input

from nbdev.showdoc import show_doc
show_doc(Source.get_files)
show_doc(Source.get_category)
show_doc(Source.get_labels)
show_doc(Source.get_extra)

<h4 id="Source.get_files" class="doc_header"><code>Source.get_files</code><a href="__main__.py#L9" class="source_link" style="float:right">[source]</a></h4>

> <code>Source.get_files</code>(**`root_dir`**:`str`)

List of paths and files to process, as returned by [`get_filenames`](/DataImporters/sources_core.html#get_filenames).

<h4 id="Source.get_category" class="doc_header"><code>Source.get_category</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>Source.get_category</code>(**`path`**:`str`, **`filename`**:`str`)

Category name for the given file (most sources hardcode a single category).

<h4 id="Source.get_labels" class="doc_header"><code>Source.get_labels</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>Source.get_labels</code>(**`path`**:`str`, **`filename`**:`str`)

List of labels for the given file.

<h4 id="Source.get_extra" class="doc_header"><code>Source.get_extra</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>Source.get_extra</code>(**`path`**:`str`, **`filename`**:`str`)

(Optional) Extra information for the given file.

## Useful functions to process the filenames

In [ ]:
#export

def normalise_label(label: str) -> str:
    """Normalise a label to be used in the metadata
    (`process` should call this automatically)."""
    return stringcase.capitalcase(stringcase.snakecase(label.lower().strip()))

def normalise_all_labels(labels: list[str]) -> list[str]:
    return [normalise_label(label) for label in labels]

In [ ]:
#export

def get_footstep_type(filename: str) -> str:
    step_types = ("walk", "scuffs", "stomps", "squishes", "wade", "scrape")
    for step in step_types:
        if step in filename.lower():
            return step
    return None

## Some other useful functions

In [ ]:
#export

def get_filenames(root_dir: str) -> list[tuple[str, str]]:
    "List audio filenames in the directory."
    return ((root, filename) for root, _, files in os.walk(root_dir)
            for filename in files
            if filename.endswith(AUDIO_EXTENSIONS))

In [ ]:
#export

def remove_all_extensions(files: list[tuple[str, str]]) -> list[tuple[str, str]]:
    return [(root, os.path.splitext(filename)[0]) for root, filename in files]

In [ ]:
#export

def get_hash(file_path: str) -> str:
    "Get the xxhash64 hash of the file."
    with open(os.path.normpath(file_path), "rb") as f:
        return xxhash.xxh64(f.read()).hexdigest()

## Processing Sources

For the most part, implementing the interface above should be enough to call `process`.

In [ ]:
#exporti

def _get_hashed_filename(path: str, filename: str) -> str:
    hash = get_hash(os.path.join(path, filename))
    extension = os.path.splitext(filename)[1]
    return hash + extension

def _prepare_target_dir(target_dir: str):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

def _sync_hashed_audio_files(files: tuple[str, str], hashes: list[str], target_dir: str):
    for [path, filename], hash in zip(files, hashes):
        original_file_path = os.path.join(path, filename)
        target_file_path = os.path.join(target_dir, hash)
        if not os.path.exists(target_file_path):
            shutil.copy2(original_file_path, target_file_path)

In [ ]:
#export

def process(source: Source, data_dir: str, version: int) -> pd.DataFrame:
    """Process source and return the metadata."""
    assert source.name is not None and source.name.strip() != ""

    source_dir = os.path.join(data_dir, "original", source.name)
    files = source.get_files(source_dir)
    if len(files) == 0:
        raise ValueError("No files found in source")

    metadata = pd.DataFrame()
    clean_files = remove_all_extensions(files)
    metadata["filename"] = hashed_filenames = [_get_hashed_filename(path, filename)
                                               for path, filename in files]
    metadata["category"] = [source.get_category(path, filename) for path, filename in clean_files]
    metadata["label"] = [",".join(normalise_all_labels(source.get_labels(path, filename)))
                         for path, filename in clean_files]
    metadata["extra"] = [source.get_extra(path, filename) for path, filename in clean_files]
    metadata["source"] = [source.name] * len(files)
    metadata["version"] = [version] * len(files)

    target_dir = os.path.join(data_dir, "intermediate", source.name)
    _prepare_target_dir(target_dir)
    
    metadata.to_csv(os.path.join(target_dir, "metadata.csv"), index=False)
    _sync_hashed_audio_files(files, hashed_filenames, target_dir)

    return metadata